## Fantasy Score Predictor

# Data Preprocessing

In [2]:
import pandas as pd

matches = pd.read_csv("Data/full_match_dat_all_countries.csv")

In [7]:
matches.head()


,Country,Date,Player,Mins,Pos,Gls,Ast,PK,PKatt,Shots,SoT,CrdY,CrdR,GA,Saves,Save%,Pk SO scorer,GK PK SO saves
0,United States,2021-09-02,Brenden Aaronson,77,"FW,MF",0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
1,United States,2021-09-02,Cristian Roldan,13,"FW,MF",0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,United States,2021-09-02,Tyler Adams,90,MF,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
3,United States,2021-09-02,Konrad De La Fuente,63,FW,0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
4,United States,2021-09-02,Kellyn Acosta,27,"DF,MF",0,0,0,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN


In [4]:
import numpy as np

np.unique(matches.loc[:,'Pos'])

array(['AM', 'AM,CM', 'AM,DM', 'AM,FW', 'AM,FW,RW', 'AM,LM', 'AM,LW',
       'AM,LW,CM', 'AM,LW,RW', 'AM,RW', 'AM,RW,RM', 'AM,WB', 'CB',
       'CB,LB', 'CB,LM', 'CB,RB', 'CM', 'CM,AM', 'CM,AM,LM', 'CM,CB',
       'CM,DM', 'CM,DM,LM', 'CM,FW', 'CM,LB', 'CM,LM', 'CM,LM,LW',
       'CM,RB,DM', 'CM,RM', 'CM,RW', 'DF', 'DF,FW', 'DF,MF', 'DM',
       'DM,CB', 'DM,CM', 'DM,LM', 'DM,RB', 'DM,RM', 'DM,RW', 'FW',
       'FW,AM', 'FW,CB', 'FW,CM', 'FW,LM', 'FW,LW', 'FW,LW,RW', 'FW,MF',
       'FW,RB', 'FW,RM', 'FW,RW', 'GK', 'LB', 'LB,CB', 'LB,FW', 'LB,LM',
       'LB,LW', 'LB,RB', 'LB,RM', 'LB,WB', 'LB,WB,LM', 'LM', 'LM,AM',
       'LM,CM', 'LM,DM', 'LM,LB', 'LM,LW', 'LM,RM', 'LM,RM,CM', 'LM,RW',
       'LM,RW,AM', 'LM,RW,RM', 'LW', 'LW,AM', 'LW,CM', 'LW,CM,DM',
       'LW,CM,LM', 'LW,FW', 'LW,LB', 'LW,LB,WB', 'LW,LM', 'LW,RM',
       'LW,RM,LM', 'LW,RM,RW', 'LW,RW', 'LW,RW,RM', 'MF', 'RB', 'RB,CB',
       'RB,LB', 'RB,RM', 'RB,RW', 'RB,WB', 'RM', 'RM,AM,LM', 'RM,CM',
       'RM,CM,AM', 'RM,DM'

In [ ]:
fantasy_pt = {
    'App':1,
    'Min':1,
    'Ast':3,
    'CrdY':-1,
    'CrdR':-3,
    'OG':-2,
    #Scrape PK won
    'PK Won':2,
    #Scrape PK conceded
    'Pen Con':-1,
    #reword GA to be passed to all players. Use GK 
    'GK DEF Clean':5,
    'Add Gls Con':-1,
    'GK Gls':9,
    'Pen Save':3,
    'Every 3 Saves':1,
    'DEF Gls':7,
    'MID Clean':1,
    'MID Gls':6,
    'Every 3 Tkls':1,
    'FWD Gls':5,
    'Every 2 FWD SoT':1
}

for i in range(len(matches)):
    pt = 0
    player = matches.loc[i,:]
    
    if player['Mins'] > 0:
        pt += fantasy_pt['App']
    if player['Mins'] >= 60:
        pt += fantasy_pt['Min']
    if player['Ast'] > 0:
        pt += player['Ast']*fantasy_pt['Ast']
    if player['CrdY'] > 0:
        pt += player['CrdY']*fantasy_pt['CrdY']
    if player['CrdR'] > 0:
        pt += player['CrdR']*fantasy_pt['CrdR']
    #this is assuming the player that took the pen is also the won who won it (not nec true)
    if player['PKatt'] > 0:
        pt += player['PKatt']*fantasy_pt['PK Won']
    #think of adding PKSave as 0 as save % if none taken record as NA need to also record number of pens attempted
    if player['PKSave'] == 0:
        pt += (player['PKatt_Opp']*(1-player['PKSave']))*fantasy_pt['PK Con']
    if player['GA'] == 0 and (player['Pos'] == 'GK' or player['Pos'] == 'DEF'):
        pt += fantasy_pt['GK DEF Clean']
    if player['GA'] >= 1:
        pt += (player['GA']-1)*fantasy_pt['Add Gls Con']
    if player['Pos'] == 'GK' and player['Gls'] >= 1:
        pt += player['Gls']*fantasy_pt['GK Gls']
    if player['PKSave'] > 0:
        pt += (player['PKatt_Opp']*player['PKSave'])*fantasy_pt['Pen Save']
    if player['Saves'] >= 3:
        saves_by3 = player['Saves']%3
        pt += saves_by3 * fantasy_pt['Every 3 Saves']
    if player['Pos'] == 'DEF' and player['Gls'] > 0:
        pt += player['Gls']*fantasy_pt['DEF Gls']   
    if player['Pos'] == 'MID' and player['GA'] == 0:
        pt += fantasy_pt['MID Clean']
    if player['Pos'] == 'MID' and player['Gls'] > 0:
        pt += player['Gls']*fantasy_pt['MID Gls']
    if player['Pos'] == 'FWD' and player['Gls'] > 0:
        pt += player['Gls']*fantasy_pt['FWD Gls']
    if player['Pos'] == 'FWD' and player['SoT'] >= 2:
        sot_by2 = player['SoT']%2
        pt += sot_by2*fantasy_pt['Every 2 FWD SoT']

9

14363